In [14]:
import pandas as pd
data= pd.read_csv('xiecheng_copy1.csv', encoding='utf8',engine='python')

FileNotFoundError: [Errno 2] No such file or directory: 'xiecheng_copy1.csv'

In [15]:
data



,lists
0,"Club Med,桂林,愚自乐园,漓江,西街,遇龙河漂流,象鼻山,两江四湖,山水间,东西巷,"
1,"桂林,两江四湖,漓江大瀑布饭店,靖江王府,阳朔,西街,遇龙河,十里画廊,银子岩,龙脊梯田,鲜..."
2,"桂林,两江四湖,漓江,阳朔,杨堤,遇龙河,十里画廊,西街,工农桥,月亮山,聚龙潭,银子岩,大..."
3,"桂林,龙脊大寨,榕杉湖景区,叠彩山,冠岩,古东瀑布,西街,漓江,杨堤,十里画廊,银子岩,世外..."
4,"容县,巴马,百魔洞,山水园,大榕树,真武阁,庆寿岩,杨贵妃故里,莲花岩,盘阳河,阳朔,西街,..."
...,...
2529,"桂林,龙胜,龙脊梯田,阳朔,遇龙河,银子岩,芦笛岩,维也纳,黄洛瑶寨,桃花江,漓江,西街,"
2530,"兴坪古镇,阳朔,"
2531,"阳朔,遇龙河,西街,世外桃源,"
2532,"桂林金水湾国际大酒店,桂林,遇龙河漂流,遇龙河,漓江,阳朔,遇龙桥,九马画山,黄布倒影,书童..."


In [13]:
#####
# 标题可视化
import jieba, re
import collections                  # 词频统计库
# 读取title，并连接成长文本
file = data.loc[:,'lists'].dropna(how='any')  # 去掉空值
string_data1 = ''.join(i for i in file)  # 把所有字符串连接成一个长文本
# 文本预处理
#pattern = re.compile(u'\t|\n|\.|-|:|;|\)|\(|\?|"') # 定义正则表达式匹配模式（空格等）
#pattern = re.compile(u'\t|\n|\.|-|:|;|\)|\(|\?|【|】[a-zA-Z0-9]"') # 定义正则表达式匹配模式（空格等）
pattern = re.compile(u'[\t|\n|\.|\-|\:|\;|\)|\(|\?|【|】|a-zA-Z0-9|\"]') # 定义正则表达式匹配模式（空格等）
string_data1 = re.sub(pattern, '', string_data1)     # 将符合模式的字符去除
# 文本分词
file_userdict = 'txtzyg.txt' #此处文件名为用户自定义的文件名，内容为不想被分开的词
jieba.load_userdict(file_userdict)
seg_list_exact = jieba.cut(string_data1, cut_all=False, HMM=True)    # 精确模式分词+HMM
object_list= []
# 去除停用词（去掉一些意义不大的词，如标点符号、嗯、啊等）
with open('stop_words.txt', 'r', encoding='UTF-8') as meaninglessFile:
    stopwords = set(meaninglessFile.read().split('\n'))
stopwords.add(' ')
for word in seg_list_exact:         # 循环读出每个分词
    if word not in stopwords:       # 如果不在去除词库中
        object_list.append(word)    # 分词追加到列表
        

# 词频统计
word_counts = collections.Counter(object_list)       # 对分词做词频统计
word_counts_top = word_counts.most_common(60)    # 获取前number个最高频的词
word_count = pd.DataFrame(word_counts_top,columns=['word', 'count'])
word_count
#包装看（整箱受欢迎>袋>礼盒）、、吃法（即食》速食（泡面燕麦那些，））
#口味上（卤味>香辣>麻辣）

FileNotFoundError: [Errno 2] No such file or directory: 'txtzyg.txt'

In [ ]:
# 数据预处理
import jieba
from sklearn.feature_extraction.text import TfidfVectorizer
x = data.title
# 停用词
stopwords = []
with open('stop_words2.txt', errors='ignore') as sf:
    for line in sf.readlines():
        stopwords.append(line.strip())
# 分词处理
def text_cut(in_text):
    words = jieba.lcut(in_text)
    cut_text = ' '.join([w for w in words if w not in stopwords and len(w) > 1])
    return cut_text
x_change = []
for i in x:
    x_change.append(text_cut(i))
# 特征提取
vectorizer = TfidfVectorizer(min_df=2, ngram_range=(1,2), strip_accents='unicode', norm='l2', token_pattern=r"(?u)\b\w+\b")
X = vectorizer.fit_transform(x_change)
# 聚类-手肘图
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
#解决中文显示问题
plt.rcParams['font.sans-serif']=['SimHei']
plt.rcParams['axes.unicode_minus'] = False
n_clusters= 70
wcss = []
for i in range(1,n_clusters):
    km = KMeans(n_clusters=i, init='k-means++', max_iter=300, n_init=10)
    km.fit(X)
    wcss.append(km.inertia_)
plt.plot(range(1,n_clusters),wcss)
plt.title('肘 部 方 法')
plt.xlabel('k(聚类的数量)')
plt.ylabel('sse')
plt.show()